In [1]:
import pandas as pd
import numpy as np
import requests 
import json
import re

In [2]:
df = pd.read_csv('./data/greenhouse.csv')

In [3]:
df.head()

,구분,신고일,철거멸실/처리 구분,철거멸실일,취소구분,취소일,존치기간,주소,연면적(㎡),용도,기타용도,구조,기타구조,동수
0,신고,20221229,NaN,NaN,NaN,NaN,20241226.0,충청남도 부여군 홍산면 조현리 471-2 외 1필지,36.00,가설건축물,저온저장고,컨테이너조,NaN,2.0
1,신고,20221229,NaN,NaN,NaN,NaN,20251229.0,충청남도 부여군 규암면 합송리 28-12,16.50,가설건축물,저온저장고,조립식판넬조,NaN,1.0
2,신고,20221229,NaN,NaN,NaN,NaN,20251225.0,충청남도 부여군 내산면 운치리 508-7,53.03,가설건축물,저온창고,컨테이너조,NaN,2.0
3,신고,20221228,NaN,NaN,NaN,NaN,20230520.0,충청남도 부여군 석성면 증산리 930-1,45.00,가설건축물,NaN,컨테이너조,NaN,2.0
4,신고,20221227,NaN,NaN,NaN,NaN,20251226.0,충청남도 부여군 남면 신홍리 323,10.40,가설건축물,저온저장고,조립식판넬조,조립식판넬조,1.0


In [4]:
# 철거일자나 취소일자가 확실하지 않은 항목들은 존치만료일자가 철거일자
# 즉 이를 실질적철거일로 복사
df['실질적철거일'] = df['존치기간']

In [5]:
# 철거일자 확실한 항목들을 실질적철거일로 지정
df.iloc[df[~pd.isna(df['철거멸실일'])].index, 14] = df.iloc[df[~pd.isna(df['철거멸실일'])].index, 3]
df.iloc[df[~pd.isna(df['철거멸실일'])].index, [3, 14]]

,철거멸실일,실질적철거일
2149,20181012.0,20181012.0
2215,20181017.0,20181017.0
2347,20190123.0,20190123.0
2412,20180409.0,20180409.0
2494,20171019.0,20171019.0
...,...,...
4353,20131124.0,20131124.0
4361,20101230.0,20101230.0
4362,20101230.0,20101230.0
4366,20120126.0,20120126.0


In [6]:
# 취소일자 확실하고 취소일자가 존치일자보다 빠른 항목들을 실질적철거일로 지정
df.iloc[(df[~pd.isna(df['취소일'])][(df[~pd.isna(df['취소일'])]['취소일'] <= df[~pd.isna(df['취소일'])]['존치기간'])]).index, 14] = df.iloc[(df[~pd.isna(df['취소일'])][(df[~pd.isna(df['취소일'])]['취소일'] <= df[~pd.isna(df['취소일'])]['존치기간'])]).index, 5]
df.iloc[(df[~pd.isna(df['취소일'])][(df[~pd.isna(df['취소일'])]['취소일'] <= df[~pd.isna(df['취소일'])]['존치기간'])]).index, [5, 14]]

,취소일,실질적철거일
36,20221209.0,20221209.0
42,20221213.0,20221213.0
66,20221215.0,20221215.0
175,20220913.0,20220913.0
193,20220808.0,20220808.0
...,...,...
4067,20211224.0,20211224.0
4107,20070828.0,20070828.0
4175,20150930.0,20150930.0
4201,20071224.0,20071224.0


In [7]:
# 주소 정제하기
df['정제주소'] = df['주소'].astype(str).map(
    lambda x: ' '.join(x.split(' ')[:-2]) if x.split(' ')[-2] == '외' else ' '.join(x.split(' ')[:])
)
df.head()

,구분,신고일,철거멸실/처리 구분,철거멸실일,취소구분,취소일,존치기간,주소,연면적(㎡),용도,기타용도,구조,기타구조,동수,실질적철거일,정제주소
0,신고,20221229,NaN,NaN,NaN,NaN,20241226.0,충청남도 부여군 홍산면 조현리 471-2 외 1필지,36.00,가설건축물,저온저장고,컨테이너조,NaN,2.0,20241226.0,충청남도 부여군 홍산면 조현리 471-2
1,신고,20221229,NaN,NaN,NaN,NaN,20251229.0,충청남도 부여군 규암면 합송리 28-12,16.50,가설건축물,저온저장고,조립식판넬조,NaN,1.0,20251229.0,충청남도 부여군 규암면 합송리 28-12
2,신고,20221229,NaN,NaN,NaN,NaN,20251225.0,충청남도 부여군 내산면 운치리 508-7,53.03,가설건축물,저온창고,컨테이너조,NaN,2.0,20251225.0,충청남도 부여군 내산면 운치리 508-7
3,신고,20221228,NaN,NaN,NaN,NaN,20230520.0,충청남도 부여군 석성면 증산리 930-1,45.00,가설건축물,NaN,컨테이너조,NaN,2.0,20230520.0,충청남도 부여군 석성면 증산리 930-1
4,신고,20221227,NaN,NaN,NaN,NaN,20251226.0,충청남도 부여군 남면 신홍리 323,10.40,가설건축물,저온저장고,조립식판넬조,조립식판넬조,1.0,20251226.0,충청남도 부여군 남면 신홍리 323


In [8]:
# 2012.01.01 이전 철거된 구조물 제거하기
df = df[df['실질적철거일'] > 20111231]

In [9]:
# 날짜형식 datetime으로 변환
df['신고일날짜'] = df['신고일'].copy().astype(int).astype(str).map(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])
df['실질적철거날짜'] = df['실질적철거일'].copy().astype(int).astype(str).map(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])

In [10]:
df = df.loc[:, ['신고일날짜', '실질적철거날짜', '정제주소', '용도', '기타용도', '구조', '동수']]
df = df.reset_index(drop=True)
df

,신고일날짜,실질적철거날짜,정제주소,용도,기타용도,구조,동수
0,2022-12-29,2024-12-26,충청남도 부여군 홍산면 조현리 471-2,가설건축물,저온저장고,컨테이너조,2.0
1,2022-12-29,2025-12-29,충청남도 부여군 규암면 합송리 28-12,가설건축물,저온저장고,조립식판넬조,1.0
2,2022-12-29,2025-12-25,충청남도 부여군 내산면 운치리 508-7,가설건축물,저온창고,컨테이너조,2.0
3,2022-12-28,2023-05-20,충청남도 부여군 석성면 증산리 930-1,가설건축물,NaN,컨테이너조,2.0
4,2022-12-27,2025-12-26,충청남도 부여군 남면 신홍리 323,가설건축물,저온저장고,조립식판넬조,1.0
...,...,...,...,...,...,...,...
4041,2001-04-25,2013-06-06,충청남도 부여군 내산면 금지리 71-2,동.식물관련시설,축사,강파이프구조,8.0
4042,2000-05-23,2012-01-25,충청남도 부여군 부여읍 석목리 148-2,제2종근린생활시설,사무실,기타강구조,1.0
4043,1999-01-11,2025-12-31,충청남도 부여군 규암면 반산리 62-3,제2종근린생활시설,농기계수리소,경량철골구조,1.0
4044,1998-04-21,2020-12-31,충청남도 부여군 홍산면 남촌리 418-4,동.식물관련시설,유리온실,일반철골구조,1.0


In [59]:
# 정제주소로 위도, 경도 계산
df_addr = df[['정제주소']]
df_addr.head()

,정제주소
0,충청남도 부여군 홍산면 조현리 471-2
1,충청남도 부여군 규암면 합송리 28-12
2,충청남도 부여군 내산면 운치리 508-7
3,충청남도 부여군 석성면 증산리 930-1
4,충청남도 부여군 남면 신홍리 323


In [60]:
API_KEY = 'c8d2487b6ee1ee531c4b6899126efe7f'
headers = {'Authorization' : 'KakaoAK '+API_KEY}
headers

{'Authorization': 'KakaoAK c8d2487b6ee1ee531c4b6899126efe7f'}

In [61]:
def getLat(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + addr
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['y'])

def getLng(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + addr
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['x'])

In [62]:
error = []
for i in df_addr.index:
    try:
        lng = getLng(df_addr['정제주소'][i])
        lat = getLat(df_addr['정제주소'][i])
        df_addr.loc[i,['경도']] = lng
        df_addr.loc[i,['위도']] = lat

    except:
        print('error',i)
        error.append(i)
        
    
    if i%100 == 0:
        print('{} 번째 주소에 대한 좌표정보 불러오는 중...'.format(i)) 

0 번째 주소에 대한 좌표정보 불러오는 중...
100 번째 주소에 대한 좌표정보 불러오는 중...
200 번째 주소에 대한 좌표정보 불러오는 중...
error 210
error 216
300 번째 주소에 대한 좌표정보 불러오는 중...
400 번째 주소에 대한 좌표정보 불러오는 중...
error 458
500 번째 주소에 대한 좌표정보 불러오는 중...
600 번째 주소에 대한 좌표정보 불러오는 중...
700 번째 주소에 대한 좌표정보 불러오는 중...
error 789
800 번째 주소에 대한 좌표정보 불러오는 중...
error 881
900 번째 주소에 대한 좌표정보 불러오는 중...
error 901
error 918
1000 번째 주소에 대한 좌표정보 불러오는 중...
1100 번째 주소에 대한 좌표정보 불러오는 중...
error 1194
1200 번째 주소에 대한 좌표정보 불러오는 중...
error 1205
error 1238
1300 번째 주소에 대한 좌표정보 불러오는 중...
1400 번째 주소에 대한 좌표정보 불러오는 중...
error 1494
error 1496
1500 번째 주소에 대한 좌표정보 불러오는 중...
error 1514
1600 번째 주소에 대한 좌표정보 불러오는 중...
error 1633
error 1668
1700 번째 주소에 대한 좌표정보 불러오는 중...
1800 번째 주소에 대한 좌표정보 불러오는 중...
error 1829
error 1833
1900 번째 주소에 대한 좌표정보 불러오는 중...
error 1960
error 1969
2000 번째 주소에 대한 좌표정보 불러오는 중...
error 2012
error 2083
2100 번째 주소에 대한 좌표정보 불러오는 중...
2200 번째 주소에 대한 좌표정보 불러오는 중...
error 2202
2300 번째 주소에 대한 좌표정보 불러오는 중...
2400 번째 주소에 대한 좌표정보 불러오는 중...
error 2456
2500 번째 주소에 대한 좌

In [70]:
df['위도'] = df_addr['위도']
df['경도'] = df_addr['경도']
df.head()

,신고일날짜,실질적철거날짜,정제주소,용도,기타용도,구조,동수,위도,경도
0,2022-12-29,2024-12-26,충청남도 부여군 홍산면 조현리 471-2,가설건축물,저온저장고,컨테이너조,2.0,36.233691,126.788844
1,2022-12-29,2025-12-29,충청남도 부여군 규암면 합송리 28-12,가설건축물,저온저장고,조립식판넬조,1.0,36.266739,126.859276
2,2022-12-29,2025-12-25,충청남도 부여군 내산면 운치리 508-7,가설건축물,저온창고,컨테이너조,2.0,36.271223,126.778802
3,2022-12-28,2023-05-20,충청남도 부여군 석성면 증산리 930-1,가설건축물,NaN,컨테이너조,2.0,36.241441,127.014539
4,2022-12-27,2025-12-26,충청남도 부여군 남면 신홍리 323,가설건축물,저온저장고,조립식판넬조,1.0,36.198915,126.791335


In [72]:
len(df)

4046

In [ ]:
df_drop = df_addr.loc[:, ['위도']].dropna(axis=0)
df = df.iloc[df_drop.index].reset_index(drop=True)

In [90]:
df = df.loc[:, ['신고일날짜', '실질적철거날짜', '용도', '구조', '위도', '경도']]
df

,신고일날짜,실질적철거날짜,용도,구조,위도,경도
0,2022-12-29,2024-12-26,가설건축물,컨테이너조,36.233691,126.788844
1,2022-12-29,2025-12-29,가설건축물,조립식판넬조,36.266739,126.859276
2,2022-12-29,2025-12-25,가설건축물,컨테이너조,36.271223,126.778802
3,2022-12-28,2023-05-20,가설건축물,컨테이너조,36.241441,127.014539
4,2022-12-27,2025-12-26,가설건축물,조립식판넬조,36.198915,126.791335
...,...,...,...,...,...,...
3975,2001-04-25,2013-06-06,동.식물관련시설,강파이프구조,36.263094,126.750682
3976,2000-05-23,2012-01-25,제2종근린생활시설,기타강구조,36.285080,126.933695
3977,1999-01-11,2025-12-31,제2종근린생활시설,경량철골구조,36.274660,126.879613
3978,1998-04-21,2020-12-31,동.식물관련시설,일반철골구조,36.208171,126.762677


In [91]:
df.to_csv('./data/preprocessed_greenhouse.csv', encoding='utf-8-sig', index=False)